In [0]:
from pyspark.sql.functions import col, when, concat_ws, to_date

df_disasters = spark.table("analytics.bronze.bronze_disasters")

In [0]:
# Checando a lista de colunas do tipo string e atribuindo à uma variável de lista.
string_columns = [f.name for f in df_disasters.schema.fields if f.dataType.typeName == 'string']

df_normalizado = df_disasters

# Criando um novo DataFrame e aplicando as transformações de lower() e trim() nos campos de string
for coluna in string_columns:
    df_normalizado = df_normalizado.withColumn(
        coluna,
        lower(trim(col(coluna)))
    )

# Removendo caracteres invisíveis 
for coluna in string_columns:
    df_normalizado = df_normalizado.withColumn(
        coluna,
        regexp_replace(col(coluna), r'[\u0000-\u001F\u007F-\u009F]','')
    )

# Exibindo o schema para validar as transformações
#df_normalizado.printSchema()
df_normalizado.limit(10).display()


In [0]:
from pyspark.sql import functions as F

# Lista de valores que indicam nulo
NULL_VALUES = ["null", "n/a", "na", "unknow", "unknown", ""]

# Função para converter colunas booleanas
def to_boolean(col_name):
    return (
        F.when(F.col(col_name).isin("yes", "y", "1", "true"), True)
         .when(F.col(col_name).isin("no", "n", "0", "false"), False)
         .otherwise(None)
    )

# Função para limpar coordenadas (latitude/longitude)
def clean_coordinate(col_name, negative_pattern):
    return F.when(F.col(col_name).isNotNull(),
                  F.when(F.col(col_name).rlike(negative_pattern),
                         -F.expr(f"try_cast(regexp_replace({col_name}, '[^0-9\\.\\-]', '') as double)"))
                   .otherwise(F.expr(f"try_cast(regexp_replace({col_name}, '[^0-9\\.\\-]', '') as double)"))
           ).otherwise(None)

df_silver = (
    df_normalizado
    # Substitui valores que indicam nulo pelo padrão do Spark (None)
    .replace(NULL_VALUES, None)

    # Conversões booleanas
    .withColumn("ofda_response", to_boolean("ofda_response"))
    .withColumn("appeal", to_boolean("appeal"))
    .withColumn("declaration", to_boolean("declaration"))

    # Conversões numéricas simples
    .withColumn("year", F.col("year").cast("int"))
    .withColumn("start_year", F.col("start_year").cast("int"))
    .withColumn("start_month", F.col("start_month").cast("int"))
    .withColumn("start_day", F.col("start_day").cast("int"))
    .withColumn("end_year", F.col("end_year").cast("int"))
    .withColumn("end_month", F.col("end_month").cast("int"))
    .withColumn("end_day", F.col("end_day").cast("int"))

    # Coordenadas geográficas
    .withColumn("latitude_clean", clean_coordinate("latitude", "(?i)[Ss]"))
    .withColumn("longitude_clean", clean_coordinate("longitude", "(?i)[Ww]"))

    # Outros campos numéricos
    .withColumn("total_deaths", F.col("total_deaths").cast("int"))
    .withColumn("no_injured", F.col("no_injured").cast("int"))
    .withColumn("no_affected", F.col("no_affected").cast("int"))
    .withColumn("no_homeless", F.col("no_homeless").cast("int"))
    .withColumn("total_affected", F.col("total_affected").cast("int"))
    .withColumn("reconstruction_costs_000_us", F.col("reconstruction_costs_000_us").cast("double"))
    .withColumn("insured_damages_000_us", F.col("insured_damages_000_us").cast("double"))
    .withColumn("total_damages_000_us", F.col("total_damages_000_us").cast("double"))

    # Datas com tratamento de erros
    .withColumn(
        "start_date",
        F.expr("try_to_timestamp(concat_ws('-', start_year, start_month, start_day))")
    )
    .withColumn(
        "end_date",
        F.expr("try_to_timestamp(concat_ws('-', end_year, end_month, end_day))")
    )

    # Padroniza país e região
    .withColumn("country", F.trim(F.initcap(F.col("country"))))
    .withColumn("region", F.trim(F.initcap(F.col("region"))))

    # Substitui latitude/longitude originais
    .drop("latitude", "longitude")
    .withColumnRenamed("latitude_clean", "latitude")
    .withColumnRenamed("longitude_clean", "longitude")
)

df_silver.printSchema()


In [0]:
df_silver.write.format("delta").mode("overwrite").saveAsTable("analytics.silver.silver_disasters")


In [0]:
from pyspark.sql import functions as F

# carregando a silver em um df
df_silver = spark.table("analytics.silver.silver_disasters")

print("Schema da tabela")
df_silver.printSchema()
print(f"Total de registros: {df_silver.count()}")

# ------------------------
# 1. Validação nulos
# ------------------------
print("\n Nulos por colunas:")
df_silver.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_silver.columns]).show(truncate=False)


# ------------------------
# 2. Datas invalidas
# ------------------------
datas_invalidas = df_silver.filter(
    (F.col("start_date") < F.lit("1900-01-01")) |
    (F.col("start_date") > F.current_date()) |
    (F.col("end_date") < F.lit("1900-01-01")) |
    (F.col("end_date") > F.current_date())
)
print(f"⏳ Linhas com datas inválidas: {datas_invalidas.count()}")

# --------------------------
# 3️ Datas invertidas
# --------------------------
datas_invertidas = df_silver.filter(F.col("end_date") < F.col("start_date"))
print(f"Linhas com datas invertidas: {datas_invertidas.count()}")

# --------------------------
# 4️ Coordenadas inválidas
# --------------------------
coords_invalidas = df_silver.filter(
    (F.col("latitude") < -90) | (F.col("latitude") > 90) |
    (F.col("longitude") < -180) | (F.col("longitude") > 180)
)
print(f"Linhas com coordenadas inválidas: {coords_invalidas.count()}")

# --------------------------
# 5 Relatório consolidado
# --------------------------
df_validacao = df_silver.withColumn(
    "erro_data_invalida",
    (F.col("start_date") < F.lit("1900-01-01")) | (F.col("start_date") > F.current_date()) |
    (F.col("end_date") < F.lit("1900-01-01")) | (F.col("end_date") > F.current_date())
).withColumn(
    "erro_coords",
    (F.col("latitude") < -90) | (F.col("latitude") > 90) |
    (F.col("longitude") < -180) | (F.col("longitude") > 180)
).withColumn(
    "erro_data_invertida",
    F.col("end_date") < F.col("start_date")
)

df_erros = df_validacao.filter(
    F.col("erro_data_invalida") | F.col("erro_coords") | F.col("erro_data_invertida")
)

print(f"\n Registros com algum erro: {df_erros.count()}")
df_erros.display(50, truncate=False)
